# Importing all necessary data
 ### Explaning the data
> - **organized_data.csv**: This file is the output of the code genereted by matrix for the correction tests correction

| nome      | cpf | idioma | cota | 1 | 2 | 3 | ... | 100 |
| --------- | --- | -------|------|---| - | - | --- | --- |
| Murilo Muniz Ozol      | 108.332.329-67  | ingles | cota_less1,5_Outros | A | B |B|...|E|
| Bruno Costa Martins   |  123.456.789-09  | espanhol | n_optante | A | B |B|...|E|

> - **gabarito.csv**: This file is the output of the work made by Hogwarts and teachers

|questao |	gabarito_ingles | gabarito_espanhol	| Matéria | Dificuldade esperada |
| ------ | ---------------- | ------------------|---------|----------------------|
|   1    |         a        |         c         | idioma  |        Fácil         |
|   2    |         a        |         d         | Física  |        Difícil       |
|   3    |         a        |         e         | idioma  |        Média         |



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from jinja2 import Environment

In [2]:
year = '2019'
test = 'Enem'
version = 'v1'

data = pd.read_csv('{0}_{1}_{2}/organized_data.csv'.format(test, year, version))
questions = pd.read_csv('{0}_{1}_{2}/gabarito.csv'.format(test, year, version), index_col=0)

In [3]:
####################### FUNCTIONS USED IN THIS SHIT ###############################
def clearParam(Param):
    cleanParam = Param.strip().capitalize()
    return cleanParam

def clearQuestionsDB(questions):
    """This function arrange the data come from Hogwarts in a way it can be used by this code"""
    # Clean the text: uppercase, remove spaces and others
    cleanEnglish = questions.loc[:,'gabarito_ingles'].str.upper().str.strip()
    cleanSpanish = questions.loc[:,'gabarito_espanhol'].str.upper().str.strip()
    cleanSubject = questions.loc[:,'Materia'].str.strip().str.capitalize()
    cleanDificulty = questions.loc[:,'Dificuldade esperada'].str.strip().str.capitalize()
    
    # Update the DataFrame
    questions.update(cleanEnglish)
    questions.update(cleanSpanish)
    questions.update(cleanSubject)
    questions.update(cleanDificulty)
    
    # Just change the variable name 
    questionsClean = questions
    
    return questionsClean

def generateTrueFalseDF(data, questionsClean):
    """This funcion compare the data and the feedback and return a pandas DataFrame with the score of each question"""
    
    # Series with the correct answer for the each languege
    englishCorrectAnswer = questionsClean.loc[:,'gabarito_ingles']
    spanishCorrectAnswer = questionsClean.loc[:,'gabarito_espanhol']
    
    # Groups divided between languege
    englishGroup = data.loc[data.idioma == 'ingles']
    spanishGroup = data.loc[data.idioma == 'espanhol']
    
    # DataFrame with just the student info
    englishStudentInfo = englishGroup.iloc[:,:englishGroup.columns.get_loc('1')]
    spanishStudentInfo = spanishGroup.iloc[:,:englishGroup.columns.get_loc('1')]
    
    # DataFrame without the student info
    englishAnswers = englishGroup.loc[:,'1':]
    spanishAnswers = spanishGroup.loc[:,'1':]
    
    # Fixing the axis for the ".eq()" funcion works
    englishCorrectAnswer.set_axis(englishAnswers.columns, inplace=True)
    spanishCorrectAnswer.set_axis(spanishAnswers.columns, inplace=True)
    
    # DataFrame with true and false for each group
    englishCorrection = englishAnswers.eq(englishCorrectAnswer)
    spanishCorrection = spanishAnswers.eq(spanishCorrectAnswer)
    
    # Join the groups of language and the student info into a DF
    englishStudentInfo = englishStudentInfo.join(englishCorrection)
    spanishStudentInfo = spanishStudentInfo.join(spanishCorrection)
    
    # Merge the two DF
    data = englishStudentInfo.append(spanishStudentInfo)
    
    #data = data.set_index('cpf')
    
    return data 

In [4]:
class Stats():
    """
    Generate all the stats for a test
    """
    
    def __init__(self, test='Enem', year='2019', version='v1'):
        """Load the test data. The user defines the test"""
        
        # Imports
        data = pd.read_csv('{0}_{1}_{2}/organized_data.csv'.format(test, year, version), index_col='cpf')
        questions = pd.read_csv('{0}_{1}_{2}/gabarito.csv'.format(test, year, version), index_col=0)
        
        # Atributes
        self.data = data
        self.questions = clearQuestionsDB(questions)
        self.trueFalseData = generateTrueFalseDF(data, self.questions)
        self.info = [self.questions, self.trueFalseData]
    
    def filtering(self, subject=None, difficulty=None, idiom=None, quota=None):
        filtered = filtering(self.trueFalseData, self.questions, subject,  difficulty, idiom, quota).filtering()
        
        return filtered
    
    def filteringBruteData(self, subject=None, difficulty=None, idiom=None, quota=None):
        filtered = filtering(self.data, self.questions, subject,  difficulty, idiom, quota).filtering()
        
        return filtered
        

In [5]:
class filtering():
    """Filtering the data (making Sant'helena happy)"""
    
    def __init__(self, trueFalseData, questions, subject, difficulty, idiom, quota):
        
        self.trueFalseData = trueFalseData
        self.questions = questions
        
        self.subject = subject
        self.difficulty = difficulty
        self.idiom = idiom
        self.quota = quota
        
    
    def filtering(self):
        
        self.questions = self.questions.groupby('Materia').get_group(self.subject) if self.subject!=None else self.questions
        self.questions = self.questions.groupby('Dificuldade esperada').get_group(self.difficulty) if self.difficulty!=None else self.questions
        self.trueFalseData = self.trueFalseData.groupby('idioma').get_group(self.idiom) if self.idiom!=None else self.trueFalseData
        self.trueFalseData = self.trueFalseData.groupby('cota').get_group(self.quota) if self.quota!=None else self.trueFalseData
        
        questionsNumber = self.questions.index.format()
        questionsNumber = pd.Series(questionsNumber).str.strip()
        dataBy = self.trueFalseData[questionsNumber]
        
        return dataBy
    

In [6]:
class subjectReport():
    """
    Doc
    """
    def __init__(self, subject, stats):
        self.subject = subject
        self.questions = stats.filtering(subject=subject)
        self.data = stats.filteringBruteData(subject=subject)
    
    def score_by_question(self, questionNumber):
        
        return self.questions.iloc[:,questionNumber-1].value_counts()

    def answers_by_question(self, questionNumber):
        
        return self.data.iloc[:,questionNumber-1].value_counts().sort_index()

In [7]:
class studentReport():
    
    def __init__(self, cpf, stats):
        self.cpf = cpf
        self.stats = stats
        self.subjects = stats.questions['Materia'].unique()
        self.dictGrades = {}
             
    @property    
    def answers(self):       
        data = self.stats.data
        studentData = pd.DataFrame(data.loc[self.cpf]).T
        studenteAnswers = studentData.loc[:,'1':]
        
        return studenteAnswers
    
    @property
    def grades(self):
        for subject in self.subjects:
            filtered = enem.filtering(subject)
            grade = filtered.loc[self.cpf].value_counts()
            self.dictGrades[subject] = grade[True]
            
        return self.dictGrades
        

In [8]:
enem =Stats()

In [15]:
student = studentReport('Cpf 50',enem)

In [16]:
biologia = subjectReport(subject='Matematica',stats = enem)

In [17]:
acertos = pd.DataFrame(index=student.grades.keys(),
             data=student.grades.values(),
             columns=['Acertos'])

## Templating with Jinja2

Here is the basics components we're going to use:
- **string_tamplate :** It's a string containing the text/html with tags showing where it's expected a variable
- **template = Enviroment() :** It's the main object of Jinja2
- **context = {} :** It's a Dict where the keys are the variable name (the same ones contained in string_tamplate) and the values are the replacement variable

```
string_tamplate = """ 
Hello, {{ name }}! How are you?
That is a simples template and I will list some things:
{% for word in words %}
> {{ word }}
{% endfor %}
"""
template = Environment().from_string(source=string_template) 
context = {
    'name' : 'Genious',
    'words' : [
        'Pizza',
        'Data',
        'Storytelling',
        'Exploratory Data Analysis',
    ],
}

rendered = template.render(context)
```

In [18]:
string_template = """
Nome:{{name}}
CPF: {{cpf}}

Idioma: {{Idiom}}
PAA: {{quota}}

Desempenho por Matéria
{% for materia in materias%}
> {{materia}}
{%endfor%}


"""

In [ ]:
template = Environment().from_string(source=string_template)

In [147]:
context = {
    'name':'Murilo',
    'cpf':'Cpf 5',
    'materias' : ['ingles',u'matemática', 'biologia'],
}

In [148]:
template.render(context)

u'\nNome:Murilo\nCPF: 108.332.329-67\n\nIdioma: \nPAA: \n\nDesempenho por Mat\xe9ria\n\n> ingles\n\n> matem\xe1tica\n\n> biologia\n\n\n'